# 기존변수에 대한 결측치 처리

#### 라이브러리 불러오기

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
smart_farm = pd.read_csv("smart_farm.csv")
columns = pd.read_csv("(스마트팜)테이블 정의서.csv")

In [3]:
smart_farm.columns = list(columns["정의"])
smart_farm.head()

,년도,월,일,시,분,일사량(W/㎡),외기온(℃),외기습도(%),풍속(m/s),풍향(º),...,차열스크린(수평),에너지스크린(수평),에너지스크린(수직),배기팬,천창,유동팬,팬코일 B동,환기온도(천창 제어온도),난방온도(설정온도),열공급량(kWh)
0,2021,11,26,0,0,-2.7,5.8,67,0.0,325,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,28.0,18.0,0.9
1,2021,11,26,0,1,-2.9,5.8,67,0.4,344,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,28.0,18.0,0.9
2,2021,11,26,0,2,-3.0,5.7,67,0.9,105,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,28.0,18.0,0.9
3,2021,11,26,0,3,-3.0,5.7,67,1.3,123,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,28.0,18.0,0.8
4,2021,11,26,0,4,-2.8,5.7,67,0.9,123,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,28.0,18.0,1.0


#### Datetime 생성하기

In [4]:
time_lst = []
ymd_lst = []

for idx, row in smart_farm.iterrows():
    date = datetime(int(row["년도"]), int(row["월"]), int(row["일"]), int(row["시"]), int(row["분"]))
    time_lst.append(date)
    
    ymd = datetime(int(row["년도"]), int(row["월"]), int(row["일"]))
    ymd_lst.append(ymd)
    
smart_farm["시간"] = time_lst
smart_farm["년월일"] = ymd_lst

In [5]:
smart_farm = smart_farm[["시간","년월일","년도",'월', '일', '시', '분', '일사량(W/㎡)', '외기온(℃)', '외기습도(%)', '풍속(m/s)',
       '풍향(º)', '내부 온도', '내부 습도', '차광스크린(수평)', '차열스크린(수평)', '에너지스크린(수평)',
       '에너지스크린(수직)', '배기팬', '천창', '유동팬', '팬코일 B동', '환기온도(천창 제어온도)',
       '난방온도(설정온도)', '열공급량(kWh)']]

#### 학습데이터와 검증데이터분리

In [6]:
smart_farm[smart_farm["시간"].duplicated()]

smart_farm_df = smart_farm.set_index("시간")

In [7]:
range1 = pd.date_range(start="2022-03-04",end="2022-03-13")
range2 = pd.date_range(start="2022-03-18",end="2022-03-23")

smart_farm_df = smart_farm_df.reset_index()

In [8]:
test_df = smart_farm_df[(smart_farm_df["년월일"].isin(range1)) | (smart_farm_df["년월일"].isin(range2))]
test_df.reset_index(drop=True,inplace=True)
# 20160개의 데이터

In [9]:
train_df = smart_farm_df[(-smart_farm_df["년월일"].isin(range1)) & (-smart_farm_df["년월일"].isin(range2))]
train_df.reset_index(drop=True,inplace=True)

#### 학습용 데이터에서 Y값 결측 제거하기

In [19]:
train_df = train_df[train_df["열공급량(kWh)"].notnull()]

#### 내부온도가 null인 512개 데이터 제거하고 다시한번 null 확인

In [20]:
train_df_2 = train_df[train_df["내부 온도"].notnull()].reset_index(drop=True)

train_df_2.isnull().sum()

시간                 0
년월일                0
년도                 0
월                  0
일                  0
시                  0
분                  0
일사량(W/㎡)           0
외기온(℃)             0
외기습도(%)            0
풍속(m/s)            0
풍향(º)              0
내부 온도              0
내부 습도              0
차광스크린(수평)          0
차열스크린(수평)          0
에너지스크린(수평)         0
에너지스크린(수직)         0
배기팬                0
천창                 0
유동팬                0
팬코일 B동             0
환기온도(천창 제어온도)    135
난방온도(설정온도)       134
열공급량(kWh)          0
dtype: int64

In [21]:
smart_farm_null = train_df_2.copy()

# Y값 결측치는 drop해주기
smart_farm_null = smart_farm_null[smart_farm_null["열공급량(kWh)"].notnull()].reset_index(drop=True)

#### test데이터에 대한 보간
- 앞뒤 평균으로 채워지지 않는 결측치 11개에 대해서는 직접 앞뒤 데이터 확인해서 보간하기

In [24]:
def get_결측치(data):
    
    측정목록 = ['내부 온도', '내부 습도', '차광스크린(수평)', '차열스크린(수평)',
       '에너지스크린(수평)', '에너지스크린(수직)', '배기팬', '천창', '유동팬', '팬코일 B동',
       '환기온도(천창 제어온도)', '난방온도(설정온도)']
    결과리스트 = []

    # 데이터프레임 copy하기
    df = data.copy()
    
    for 항목 in 측정목록:
        for idx,row in df[df[항목].isnull()].iterrows():
            if df.loc[idx+1,항목] != np.nan:
                if df.loc[idx-1,항목] != np.nan:
                    df.loc[idx,항목] = ( df.loc[idx+1,항목]+df.loc[idx-1,항목] ) / 2
    결과리스트.append(df)
    
    return pd.concat(결과리스트,axis=0,ignore_index = True)

In [25]:
test_df_nonull = get_결측치(test_df)

In [26]:
test_df_nonull.to_csv("test_df.csv",encoding="utf-8-sig",index=False)

#### 결측치 직접 처리하기...
- test_df_null.csv : 결측치 직접 처리한 데이터
- 환기온도만 결측치가 271개 정도 존재함


- 연속된 시간에 존재하는 3월7일, 3월8일을 제외하고 나머지는 평균으로 보간하기
- 3월 8일 11시 4분 ~ 3월 8일 11시 23분 앞뒤의 환기온도가 모두 30도 이기에 30도로 보간하기
- 환기온도 데이터의 특성과 앞뒤 데이터를 적절하게 판단해서 보간한 것

In [27]:
test_df_nonull2 = pd.read_csv("test_df_null.csv",encoding="cp949")

test_df_nonull2["시간"] = test_df_nonull["시간"]
test_df_nonull2["년월일"] = test_df_nonull["년월일"]

In [28]:
def get_온도_결측치(data):
    
    측정목록 = ["환기온도(천창 제어온도)","난방온도(설정온도)"]
    결과리스트 = []

    # 데이터프레임 copy하기
    df = data.copy()
    
    for 항목 in 측정목록:
        for idx,row in df[df[항목].isnull()].iterrows():
            if df.loc[idx+1,항목] != np.nan:
                if df.loc[idx-1,항목] != np.nan:
                    df.loc[idx,항목] = ( df.loc[idx+1,항목]+df.loc[idx-1,항목] ) / 2
    결과리스트.append(df)
    
    return pd.concat(결과리스트,axis=0,ignore_index = True)

In [29]:
smart_farm_nonull = get_온도_결측치(smart_farm_null)

In [30]:
print("결측치 처리 이전",smart_farm_null.isnull().sum().sum())
print("결측치 처리 이후",smart_farm_nonull.isnull().sum().sum())

결측치 처리 이전 269
결측치 처리 이후 231


In [32]:
# 결측치 처리 전
smart_farm_null.iloc[[5282,5283,5284,5492,5493,5495],:]

,시간,년월일,년도,월,일,시,분,일사량(W/㎡),외기온(℃),외기습도(%),...,차열스크린(수평),에너지스크린(수평),에너지스크린(수직),배기팬,천창,유동팬,팬코일 B동,환기온도(천창 제어온도),난방온도(설정온도),열공급량(kWh)
5282,2021-11-29 17:32:00,2021-11-29,2021,11,29,17,32,-2.7,13.4,44,...,100.0,0.0,0.0,0.0,0.0,1.0,0.0,27.0,18.4,0.0
5283,2021-11-29 17:34:00,2021-11-29,2021,11,29,17,34,-2.7,13.4,44,...,100.0,0.0,0.0,0.0,0.0,1.0,0.0,NaN,NaN,0.0
5284,2021-11-29 17:36:00,2021-11-29,2021,11,29,17,36,-2.7,13.4,45,...,100.0,0.0,0.0,0.0,0.0,1.0,0.0,27.0,18.2,0.0
5492,2021-11-29 21:04:00,2021-11-29,2021,11,29,21,4,-2.8,6.3,72,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,28.0,16.7,0.5
5493,2021-11-29 21:05:00,2021-11-29,2021,11,29,21,5,-2.8,6.3,72,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,NaN,1.5
5495,2021-11-29 21:07:00,2021-11-29,2021,11,29,21,7,-3.0,6.3,72,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,28.0,16.8,0.2


In [33]:
# 결측치 처리 후
smart_farm_nonull.iloc[[5282,5283,5284,5492,5493,5495],:]

,시간,년월일,년도,월,일,시,분,일사량(W/㎡),외기온(℃),외기습도(%),...,차열스크린(수평),에너지스크린(수평),에너지스크린(수직),배기팬,천창,유동팬,팬코일 B동,환기온도(천창 제어온도),난방온도(설정온도),열공급량(kWh)
5282,2021-11-29 17:32:00,2021-11-29,2021,11,29,17,32,-2.7,13.4,44,...,100.0,0.0,0.0,0.0,0.0,1.0,0.0,27.0,18.40,0.0
5283,2021-11-29 17:34:00,2021-11-29,2021,11,29,17,34,-2.7,13.4,44,...,100.0,0.0,0.0,0.0,0.0,1.0,0.0,27.0,18.30,0.0
5284,2021-11-29 17:36:00,2021-11-29,2021,11,29,17,36,-2.7,13.4,45,...,100.0,0.0,0.0,0.0,0.0,1.0,0.0,27.0,18.20,0.0
5492,2021-11-29 21:04:00,2021-11-29,2021,11,29,21,4,-2.8,6.3,72,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,28.0,16.70,0.5
5493,2021-11-29 21:05:00,2021-11-29,2021,11,29,21,5,-2.8,6.3,72,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,28.0,16.75,1.5
5495,2021-11-29 21:07:00,2021-11-29,2021,11,29,21,7,-3.0,6.3,72,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,28.0,16.80,0.2


In [34]:
smart_farm_nonull.to_csv("train_df.csv", encoding="utf-8-sig", index=False)

#### 결측치 직접 처리하기...
- train_df.csv : 결측치 직접 처리할 데이터

- train_df_null.csv : 결측치 직접 처리완료한 데이터

In [35]:
train_df_null = pd.read_csv("train_df_null.csv",encoding="cp949")

In [36]:
train_df_null.isnull().sum()

시간                0
년월일               0
년도                0
월                 0
일                 0
시                 0
분                 0
일사량(W/㎡)          0
외기온(℃)            0
외기습도(%)           0
풍속(m/s)           0
풍향(º)             0
내부 온도             0
내부 습도             0
차광스크린(수평)         0
차열스크린(수평)         0
에너지스크린(수평)        0
에너지스크린(수직)        0
배기팬               0
천창                0
유동팬               0
팬코일 B동            0
환기온도(천창 제어온도)    37
난방온도(설정온도)       65
열공급량(kWh)         0
dtype: int64

#### 난방온도 결측치 존재하는 것 다 제외
- 난방온도는 적절한 보간이 어렵고, 충분히 개수가 되지 않기에 제거

In [37]:
train_df_null = train_df_null[train_df_null["난방온도(설정온도)"].notnull()]

In [38]:
train_df_null.isnull().sum()

시간               0
년월일              0
년도               0
월                0
일                0
시                0
분                0
일사량(W/㎡)         0
외기온(℃)           0
외기습도(%)          0
풍속(m/s)          0
풍향(º)            0
내부 온도            0
내부 습도            0
차광스크린(수평)        0
차열스크린(수평)        0
에너지스크린(수평)       0
에너지스크린(수직)       0
배기팬              0
천창               0
유동팬              0
팬코일 B동           0
환기온도(천창 제어온도)    0
난방온도(설정온도)       0
열공급량(kWh)        0
dtype: int64

In [39]:
test_df_nonull2.isnull().sum()

시간                   0
년월일                  0
년도                   0
월                    0
일                    0
시                    0
분                    0
일사량(W/㎡)             0
외기온(℃)               0
외기습도(%)              0
풍속(m/s)              0
풍향(º)                0
내부 온도                0
내부 습도                0
차광스크린(수평)            0
차열스크린(수평)            0
에너지스크린(수평)           0
에너지스크린(수직)           0
배기팬                  0
천창                   0
유동팬                  0
팬코일 B동               0
환기온도(천창 제어온도)      241
난방온도(설정온도)           0
열공급량(kWh)        18720
dtype: int64

In [40]:
train_test_df = pd.concat([train_df_null,test_df_nonull2],axis=0,ignore_index=True)

#### 분류대체 방법
- 예측용 데이터에 있는 환기온도 241개 데이터 결측치 해소
- 범주형 값으로 매핑

In [41]:
train_test_df_copy = train_test_df.copy()

In [42]:
train_test_df_copy.isnull().sum()

시간                   0
년월일                  0
년도                   0
월                    0
일                    0
시                    0
분                    0
일사량(W/㎡)             0
외기온(℃)               0
외기습도(%)              0
풍속(m/s)              0
풍향(º)                0
내부 온도                0
내부 습도                0
차광스크린(수평)            0
차열스크린(수평)            0
에너지스크린(수평)           0
에너지스크린(수직)           0
배기팬                  0
천창                   0
유동팬                  0
팬코일 B동               0
환기온도(천창 제어온도)      241
난방온도(설정온도)           0
열공급량(kWh)        18720
dtype: int64

In [43]:
def to_categorical(x):
    if 27 <= x < 27.5:
        return 27
    elif 27.5 <= x < 29:
        return 28
    elif 29 <= x:
        return 30  

In [44]:
# 27, 28, 30으로 매핑하기
train_test_df_copy["환기온도(천창 제어온도)"] = train_test_df_copy["환기온도(천창 제어온도)"].apply(lambda x : to_categorical(x))

#### 환기온도 결측
- 회귀대체로 처리하기

In [45]:
train_test_df_copy_all_nonull = train_test_df_copy.dropna(axis=0,how="any")

vent_temp = train_test_df_copy[train_test_df_copy["환기온도(천창 제어온도)"].isnull()]
# vent_temp(환기 온도) : 환기온도(천창 제어온도)만 null인 데이터

In [46]:
col_lst = ['일사량(W/㎡)', '외기온(℃)', '외기습도(%)',
       '풍속(m/s)', '풍향(º)', '내부 온도', '내부 습도', '차광스크린(수평)', '차열스크린(수평)',
       '에너지스크린(수평)', '에너지스크린(수직)', '배기팬', '천창', '유동팬', '팬코일 B동',
                   '난방온도(설정온도)',"열공급량(kWh)"]

X = train_test_df_copy_all_nonull[col_lst]

y = train_test_df_copy_all_nonull["환기온도(천창 제어온도)"]

In [47]:
import lightgbm as lgbm
clf = lgbm.LGBMClassifier()
clf.fit(X=X,y=y)
predicted = clf.predict(vent_temp[col_lst])

In [48]:
predicted

array([30., 30., 30., 30., 30., 30., 30., 30., 30., 30., 30., 30., 30.,
       30., 30., 30., 30., 30., 30., 30., 27., 30., 30., 30., 30., 30.,
       30., 30., 30., 30., 30., 30., 30., 30., 30., 30., 30., 30., 30.,
       30., 30., 30., 30., 30., 30., 30., 30., 27., 27., 27., 27., 30.,
       27., 27., 27., 27., 27., 27., 27., 27., 27., 27., 27., 27., 27.,
       27., 27., 27., 27., 27., 27., 27., 27., 27., 27., 27., 27., 27.,
       27., 27., 27., 27., 27., 27., 27., 27., 27., 27., 27., 27., 27.,
       27., 27., 27., 27., 27., 27., 27., 27., 27., 27., 27., 27., 27.,
       27., 27., 27., 27., 27., 27., 27., 27., 27., 27., 27., 27., 27.,
       27., 27., 27., 27., 27., 27., 27., 27., 27., 27., 27., 27., 27.,
       27., 27., 27., 27., 27., 27., 27., 27., 27., 27., 27., 27., 27.,
       27., 27., 27., 27., 27., 27., 27., 27., 27., 27., 27., 27., 27.,
       27., 27., 27., 27., 27., 27., 27., 27., 27., 27., 27., 27., 27.,
       27., 27., 27., 27., 27., 27., 27., 27., 27., 27., 27., 27

In [49]:
vent_temp['환기온도(천창 제어온도)'] = predicted

<ipython-input-49-cc3e098fc56e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vent_temp['환기온도(천창 제어온도)'] = predicted


In [50]:
# 예측한 값들 채우기
train_test_df_copy.loc[vent_temp.index,"환기온도(천창 제어온도)"] = train_test_df_copy.loc[vent_temp.index,
                                                   "환기온도(천창 제어온도)"].fillna(vent_temp["환기온도(천창 제어온도)"])

In [51]:
train_test_df_copy.iloc[:130886,:].to_csv("smart_farm_train.csv",encoding="utf-8-sig",index=False)

In [52]:
train_test_df_copy.iloc[130886:,:].reset_index(drop=True).to_csv("smart_farm_test.csv",encoding="utf-8-sig",index=False)

------------------------------------------------------------ 구분선 ---------------------------------------------------------------------